# pair paragraphs

In [263]:
is_pl = True
par = {}
pars = []
for line in open('text'):
    k = 'pl' if is_pl else 'en'
    par[k] = par.get(k, [])
    if line == '\n':
        par[k] = " ".join(par[k])
        
        if not is_pl:
            # the pars go: pl, en, pl, en
            pars.append(par)
            par = {}
            
        is_pl = not is_pl
        continue
    par[k].append(line.strip())
    
import json

with open('pars', 'w') as f:
    for par in pars:
        f.write(json.dumps(par).replace('…','...'))
        f.write('\n')

# pair sentences

In [265]:
def fix(x):
    return x.replace('…', '.').replace('Mrs.', 'Mrs').replace('Mr.', 'Mr')

def fix2(x):
    return [i for i in x if i]

a = 0
with open('sen', 'w') as f:
    b = 0
    for par in open('pars'):
        b = b + 1
        par = json.loads(fix(par))
        en = fix2(par['en'].split('.'))
        pl = fix2(par['pl'].split('.'))
        
        if len(pl) != len(en):
            a = a + 1
            print('------')
            [print(i) for i in pl]
            print('---')
            [print(i) for i in en]
            break
        
        d = {
            'en': en,
            'pl'
        }
        f.write()


In [262]:
a

1

In [251]:
en

['Agnes couldn’t believe it',
 ' It was like a dream come true',
 ' Since that day, her idol was her best friend',
 ' Agnes was at every concert of his and spent time with him',
 ' She always had a place backstage']